In [243]:
import pandas as pd
import datetime
df = pd.read_csv( "castle-event-data-v4.csv",sep=',', header=0, parse_dates=True,quoting=1 )


In [244]:
import numpy as np

#### replace all values "custome" in column name with custom name

In [245]:
df.name=df.name.replace(to_replace='custom',value=np.nan)
df.name=df.name.fillna(df.custom_name)

#### read in the agent os file i generated from user agent API

In [246]:
def agent_name_from_api(ua_unique,file_name='agent_name.csv'):
    i=0    
    if not os.path.isfile(file_name):
        for ua in ua_unique:
            if i%500==0:
                print('finishing {0} out of {1} agent'.format(i,len(ua_unique)))
            user_agen_api = requests.get("http://www.useragentstring.com/?uas="+ua+'&getJSON=os_name-agent_name')
            if user_agen_api.status_code==200:
                user_agent[ua]=[user_agen_api.json()['os_name'],user_agen_api.json()['agent_name']]
            i+=1
        df_ua=pd.DataFrame.from_dict(user_agent,orient='index',)
#        df_ua.column=['user_agent']
        df_ua.to_csv('agent_name.csv')
    else: df_ua=pd.read_csv('agent_name.csv')
    return df_ua

In [247]:
#read in the agent os file i generated from user agent API
df_agent_os=pd.read_csv('agent_name.csv')

In [248]:
df_agent_os.columns.values

array(['Unnamed: 0', '0', '1'], dtype=object)

In [249]:
df_agent_os.rename(columns={'Unnamed: 0':'ua','0':'os','1':'agent_name'},inplace=True)

In [250]:
df_agent_os.head()

,ua,os,agent_name
0,Mozilla/5.0 (Linux; Android 5.1.1; Ilium_PAD_i...,Android,Android Webkit Browser
1,Mozilla/5.0 (Linux; Android 4.4.2; SM-G355M Bu...,Android,Android Webkit Browser
2,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,Windows NT,Internet Explorer
3,Mozilla/5.0 (Linux; Android 5.0.1; SM-N910P Bu...,Android,Android Webkit Browser
4,Mozilla/5.0 (Linux; Android 4.4.4; ILIUM L900 ...,Android,Android Webkit Browser


In [251]:
df=df.merge(df_agent_os,left_on='ua',right_on='ua',how='left')

####  Calculate the percentile of each time difference between actions

In [252]:
df.sort_values(by=['device_id','created_at'],ascending=True,inplace=True)
device_grouped=df.groupby(by='device_id')
df['time_diff']=device_grouped.created_at.diff()

In [253]:
df.time_diff=df.time_diff.fillna(df.time_diff.mean())

In [254]:
df['time_diff_pct']=df.time_diff.rank(pct=True)

In [255]:
df.columns

Index([u'created_at', u'user_id', u'session_id', u'device_id', u'ua', u'l',
       u'name', u'custom_name', u'ip', u'continent', u'country', u'city',
       u'lon', u'lat', u'accuracy', u'isp_name', u'datacenter', u'proxy',
       u'mobile_device', u'os', u'agent_name', u'time_diff', u'time_diff_pct'],
      dtype='object')

#### Calculate the speed ofuser travels

In [256]:
df['lon_diff']=device_grouped.lon.diff()
df['lat_diff']=device_grouped.lat.diff()

In [257]:
df['speed']=(df.lon_diff.pow(2)+df.lat_diff.pow(2))/df.time_diff

In [258]:
df.speed=df.speed.fillna(0)

#### Calculate the percentile of each time difference between actions, country, isp_name, agent_name

In [259]:
# function to convert categorical to percentile
def cat_to_pct(df,col):
    pct=df[col].value_counts(dropna=False)/df[col].notnull().sum()
    df=df.join(pct,on=[col],how='left',rsuffix='_pct')
    return df

In [260]:
df=cat_to_pct(df,'os')

In [261]:
df=cat_to_pct(df,'country')

In [262]:
df=cat_to_pct(df,'continent')

In [263]:
df=cat_to_pct(df,'isp_name')

In [264]:
df=cat_to_pct(df,'agent_name')

In [265]:
df=cat_to_pct(df,'name')

In [184]:
df.columns

Index([u'created_at', u'user_id', u'session_id', u'device_id', u'ua', u'l',
       u'name', u'custom_name', u'ip', u'continent', u'country', u'city',
       u'lon', u'lat', u'accuracy', u'isp_name', u'datacenter', u'proxy',
       u'mobile_device', u'os', u'agent_name', u'time_diff', u'time_diff_pct',
       u'country_pct', u'name_pct', u'os_pct', u'isp_name_pct',
       u'agent_name_pct', u'lon_diff', u'lat_diff', u'speed',
       u'continent_pct'],
      dtype='object')

#### Fillna to remove all missing values

In [223]:
df.os_pct=df.os_pct.fillna(df.os_pct.mean())

In [226]:
df.isp_name_pct=df.isp_name_pct.fillna(df.isp_name_pct.mean())

In [234]:
df.continent_pct=df.continent_pct.fillna(df.continent_pct.mean())

In [ ]:
df.continent_pct=df.continent_pct.fillna(df.continent_pct.mean())

In [269]:
device_id=df.device_id.copy()
df_model=df.drop(['created_at','user_id','session_id','device_id','ua','l','name','custom_name'\
                  ,'ip','continent','country','city','lon','lat','accuracy','isp_name','os','agent_name'\
                  ,'lon_diff','lat_diff'],axis=1)
#df_model=df_model.drop(['l','city','isp_name'],axis=1)

In [270]:
df_model.head()

,datacenter,proxy,mobile_device,time_diff,time_diff_pct,speed,os_pct,country_pct,continent_pct,isp_name_pct,agent_name_pct,name_pct
1107188,False,False,True,3.796006e+07,0.826382,0.0,0.328734,0.890437,0.891544,0.106760,0.318727,0.727196
44217,False,False,False,3.796006e+07,0.826382,0.0,0.017149,0.890437,0.891544,0.000098,0.016912,0.159076
28223,False,False,False,1.025682e+09,0.992641,0.0,0.017149,0.890437,0.891544,0.000098,0.016912,0.159076
453132,False,False,False,3.796006e+07,0.826382,0.0,0.018319,0.890437,0.891544,0.129371,0.072780,0.727196
453133,False,False,False,4.976000e+03,0.091810,0.0,0.018319,0.890437,0.891544,0.129371,0.072780,0.727196


In [271]:
df_model.describe()

,time_diff,time_diff_pct,speed,os_pct,country_pct,continent_pct,isp_name_pct,agent_name_pct,name_pct
count,2.111004e+06,2.111004e+06,2.111004e+06,2.111004e+06,2.111004e+06,2.111004e+06,2.111004e+06,2.111004e+06,2.111004e+06
mean,3.796006e+07,5.000002e-01,1.872203e-04,2.327306e-01,8.016245e-01,8.052155e-01,6.223148e-02,3.394260e-01,5.565489e-01
std,1.573715e+08,2.881391e-01,1.189136e-01,1.074913e-01,2.527332e-01,2.469724e-01,5.247955e-02,1.595239e-01,2.808946e-01
min,0.000000e+00,7.532672e-03,0.000000e+00,1.894868e-06,4.739806e-07,8.294562e-05,4.739941e-07,4.737082e-07,2.368541e-06
25%,1.698300e+04,2.499986e-01,0.000000e+00,1.870202e-01,8.904370e-01,8.915436e-01,2.519279e-03,3.187275e-01,1.590760e-01
50%,6.730800e+04,5.000002e-01,0.000000e+00,2.852141e-01,8.904370e-01,8.915436e-01,6.722421e-02,3.187275e-01,7.271962e-01
75%,3.796006e+07,8.263819e-01,0.000000e+00,3.287340e-01,8.904370e-01,8.915436e-01,1.285571e-01,4.754373e-01,7.271962e-01
max,2.658709e+09,1.000000e+00,1.495481e+02,3.287340e-01,8.904370e-01,8.915436e-01,1.293715e-01,4.754373e-01,7.271962e-01


In [275]:
df_model[df_model.isp_name_pct.isnull()]

,datacenter,proxy,mobile_device,time_diff,time_diff_pct,speed,os_pct,country_pct,continent_pct,isp_name_pct,agent_name_pct,name_pct


In [195]:
#initialize isolation forest
from sklearn.ensemble import IsolationForest
seed=25
sf=IsolationForest(contamination=0.001,n_jobs=-1,random_state=seed)

In [276]:
# fitting the isolation forest model
sf.fit(df_model)


IsolationForest(bootstrap=False, contamination=0.001, max_features=1.0,
        max_samples='auto', n_estimators=100, n_jobs=-1, random_state=25,
        verbose=0)

In [278]:
predict_anom=sf.predict(df_model)

In [279]:
df_model['score']=predict_anom

In [280]:
df['score']=predict_anom

In [ ]:
df_trans.head()

In [301]:
df_anom=df[df.score==-1]

In [302]:
df_anom.columns

Index([u'created_at', u'user_id', u'session_id', u'device_id', u'ua', u'l',
       u'name', u'custom_name', u'ip', u'continent', u'country', u'city',
       u'lon', u'lat', u'accuracy', u'isp_name', u'datacenter', u'proxy',
       u'mobile_device', u'os', u'agent_name', u'time_diff', u'time_diff_pct',
       u'lon_diff', u'lat_diff', u'speed', u'os_pct', u'country_pct',
       u'continent_pct', u'isp_name_pct', u'agent_name_pct', u'name_pct',
       u'score'],
      dtype='object')

In [303]:
df_anom.iloc[:,:20].head()

,created_at,user_id,session_id,device_id,ua,l,name,custom_name,ip,continent,country,city,lon,lat,accuracy,isp_name,datacenter,proxy,mobile_device,os
1491583,1470711576266,5794748,70905446,7890958,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,NaN,page_viewed,NaN,207.244.83.106,North America,United States,Manassas,-77.6321,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 10
1476129,1470874241030,5794748,72867752,7890958,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,NaN,page_viewed,NaN,207.244.83.106,North America,United States,Manassas,-77.6321,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 10
885546,1470663712913,5794942,70157080,7891230,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,NaN,page_viewed,NaN,207.244.93.228,North America,United States,Manassas,-77.6321,38.7701,1000.0,Leaseweb USA,True,False,False,Windows NT
63465,1470779183688,5794942,71720937,7891230,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,NaN,page_viewed,NaN,174.34.164.122,North America,United States,Chicago,-87.6248,41.8471,1000.0,"Nobis Technology Group, LLC",True,False,False,Windows NT
63477,1470779805457,5794942,71720937,7891230,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,NaN,page_viewed,NaN,207.244.77.13,North America,United States,Manassas,-77.6321,38.7701,1000.0,Leaseweb USA,True,False,False,Windows NT


In [285]:
# columns in model: 'name', u'continent', u'country', u'datacenter', u'proxy',
#       u'mobile_device'
df_anom.name.value_counts(dropna=False)

page_viewed                                   1479
login_succeeded                                482
Payment With Balance                            37
Mobile Show Cvv                                 21
passwordReset_requested                         19
Cellphone Confirmation                          15
passwordChange_succeeded                        10
Account Create Edc Deposit On Landing Page       9
First Card Access                                9
registration_succeeded                           9
First Timeline Access                            8
New Contact                                      4
Create Withdraw                                  4
Regular Cellphone Confirmation                   4
Account Login Soft Block                         1
Mobile Withdraw Check                            1
Name: name, dtype: int64

In [286]:
df.name.value_counts()

page_viewed                                   1535114
login_succeeded                                335810
passwordChange_succeeded                        73548
Payment With Balance                            62531
Regular Cellphone Confirmation                  26639
passwordReset_requested                         14829
Create Withdraw                                 13383
Account Create Edc Deposit On Landing Page      12417
registration_succeeded                          11175
Card Checkout                                    7152
Mobile Show Cvv                                  4975
Cellphone Confirmation                           3864
First Timeline Access                            3580
First Card Access                                3576
Mobile Withdraw Check                            1728
New Contact                                       463
Account Login Soft Block                          200
Mobile Account Change Password                      9
Regular Cellphone Confirmati

In [305]:
anom_gp=df_anom.groupby(by='device_id').created_at.count()

In [306]:
anom_gp.sort_values(ascending=False)

device_id
8066019     22
7949210     21
8302850     20
8631904     19
8737334     19
8013668     19
8019509     16
7927147     16
8283694     15
10351834    15
8010650     14
9062420     14
10058108    14
7975122     13
9557890     12
11202414    12
8651184     12
10451762    12
8083873     12
8155942     11
9092303     11
8459576     11
8260962     11
9275580     11
7892262     11
9220651     10
9154793     10
9247039     10
9747715     10
8070380      9
            ..
10568173     1
8409346      1
8409814      1
8410064      1
10544345     1
8411618      1
10540049     1
8413348      1
8417826      1
10535393     1
8418278      1
10531560     1
8426491      1
10522025     1
8451623      1
8451762      1
8452828      1
10495434     1
8463046      1
8468400      1
10488626     1
10480774     1
10470296     1
8608283      1
10456820     1
10405416     1
10366589     1
8674414      1
8676659      1
13119681     1
Name: created_at, dtype: int64

In [309]:
df[df.device_id==8066019 ].iloc[:,-20:]

,lat,accuracy,isp_name,datacenter,proxy,mobile_device,os,agent_name,time_diff,time_diff_pct,lon_diff,lat_diff,speed,os_pct,country_pct,continent_pct,isp_name_pct,agent_name_pct,name_pct,score
836237,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 7,Chrome,3.796006e+07,0.826382,NaN,NaN,0.000000e+00,0.285214,0.008106,0.103950,0.002771,0.475437,0.727196,1
981543,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 7,Chrome,1.137300e+04,0.182283,0.0000,0.0000,0.000000e+00,0.285214,0.008106,0.103950,0.002771,0.475437,0.159076,1
836240,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 7,Chrome,2.813300e+04,0.347886,0.0000,0.0000,0.000000e+00,0.285214,0.008106,0.103950,0.002771,0.475437,0.727196,1
836263,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 7,Chrome,3.658140e+05,0.633540,0.0000,0.0000,0.000000e+00,0.285214,0.008106,0.103950,0.002771,0.475437,0.727196,1
836319,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 7,Chrome,7.476810e+05,0.661772,0.0000,0.0000,0.000000e+00,0.285214,0.008106,0.103950,0.002771,0.475437,0.727196,1
828855,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 7,Chrome,2.657510e+05,0.617219,0.0000,0.0000,0.000000e+00,0.285214,0.008106,0.103950,0.002771,0.475437,0.727196,1
827667,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 7,Chrome,6.827957e+07,0.921334,0.0000,0.0000,0.000000e+00,0.285214,0.008106,0.103950,0.002771,0.475437,0.727196,1
977750,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 7,Chrome,3.535000e+03,0.068645,0.0000,0.0000,0.000000e+00,0.285214,0.008106,0.103950,0.002771,0.475437,0.159076,1
827668,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 7,Chrome,1.136000e+04,0.182113,0.0000,0.0000,0.000000e+00,0.285214,0.008106,0.103950,0.002771,0.475437,0.727196,1
827671,38.7701,1000.0,Leaseweb USA,True,False,False,Windows 7,Chrome,3.478200e+04,0.389051,0.0000,0.0000,0.000000e+00,0.285214,0.008106,0.103950,0.002771,0.475437,0.727196,1
